Let's make that last visualization look nicer. We're still using the [Aerial Waterfowl Survey Data](https://data.delaware.gov/Energy-and-Environment/Aerial-Waterfowl-Survey-Data/bxyv-7mgn). 


In [ ]:
# Import our libraries
import pandas as pd

import plotly.offline as offline
import plotly.graph_objs as go

# We're using json this time!
import json

# Run this in offline mode
offline.init_notebook_mode()


In [ ]:
# Run the transformations from before to make sure everyone is 
# at the same place.
url = "https://data.delaware.gov/api/views/bxyv-7mgn/rows.csv?accessType=DOWNLOAD"
waterfowl_df = pd.read_csv(url)
waterfowl_df_january = waterfowl_df[waterfowl_df['Month']=='January']
waterfowl_df_january_sub = waterfowl_df_january[waterfowl_df_january['Time Period']!='Late']

waterfowl_df_january_sub_by_year = waterfowl_df_january_sub.groupby('Year').sum()

years = [str(year) for year in waterfowl_df_january_sub_by_year.index]


In [ ]:
# Now let's plot the top three.
# This is creating the same chart as before, but options to go.Scatter each have their own line and 
# the layout is being created from a dictionary (our first step toward displaying in Javascript!)

bird_names = ['Canada Goose', 'American Black Duck', 'Mallard']

data = []

for bird_name in bird_names:
    
    single_bird = waterfowl_df_january_sub[['Year', bird_name]].groupby('Year').sum()

    bird_counts = [int(total) for total in single_bird[bird_name]]
    
    # Cheat and re-usse the years variable from before
    data.append(
        go.Scatter(
            x=years, 
            y=bird_counts, 
            mode="markers+lines", 
            name=bird_name
        )
    )

layout_dict = {
    'title': "Top three birds", 
    'xaxis':{'title':'Year'}, 
    'yaxis':{'title':'Number counted'}
}

layout=go.Layout(layout_dict)


figure=go.Figure(data=data,layout=layout)

offline.iplot(figure, filename='top_three')

### Things to think about:

Before diving into Plotly's documentation, is there anything in the block above that you want to edit and test?

How can you make the Title more descriptive? 

How can you edit the order of the birds in the legend on the right of the graph?

Notice how mode is set to "markers+lines"? What happens if you make it just "markers" or "lines"?

How would you change the hover text? Seeing "123.431K" looks weird (Canada Goose, 2001)



### How to add your own flair to the chart above

To see what options there are to make Plotly look nicer, checkout [the documentation for Scatter](https://plot.ly/python/line-and-scatter/). 

There's a lot there! And it's hard to picture how most of the options will change the graph... 

I recommend looking at these options: hoverinfo, hovertext, line, marker


"Style" is inherently subjective, but to get a feel for best practices consult a style guide. Personally I like the [Sunlight Foundation's](https://github.com/amycesal/dataviz-style-guide/blob/master/Sunlight-StyleGuide-DataViz.pdf), but it's a PDF. This one by the [Uban Institute](http://urbaninstitute.github.io/graphics-styleguide/) is a good one to read in a browser.

Note: We're jumping ahead to just the variables for Scatter. To get a general overview of ploty, read the [user guide](https://plot.ly/python/user-guide/).

When you're ready, explore the ['Basic Charts' section of the Python documentation](https://plot.ly/python/).


### Make your changes!

Below is the same chart as above, but the code is different! The line chart options are now using a Python dictionary 
just like the layout options above. I also made the chart name more descriptive. The format() method replaces the '{}' placeholders in the text with variables passed to it. [Learn more about .format() in the documentation](https://docs.python.org/2/library/string.html#format-string-syntax).

Note the addtion of a new chart option in the dictionary. This will be needed by the Javascript: 'type':'scatter'


In [ ]:
# ***** This cell requires you to fill something in! *****

# Edit this function to change the style of the chart!

def return_data_dictionary(years, bird_counts, bird_name):
    return {
        # Add to these options to make your own chart
        
        'type':'scatter',
        'x':years, 
        'y':bird_counts, 
        'mode':"markers+lines", 
        'name':bird_name
    }

bird_names = ['Canada Goose', 'American Black Duck', 'Mallard']

# We'll use this variable to create our Javascript graph, but first we'll use it create our Python one!
pre_json_dictionary = {
    'data': []
}

for bird_name in bird_names:
    
    single_bird = waterfowl_df_january_sub[['Year', bird_name]].groupby('Year').sum()

    bird_counts = [int(total) for total in single_bird[bird_name]]
    
    # Append our dictionary to the data list
    pre_json_dictionary['data'].append(return_data_dictionary(years, bird_counts, bird_name))


pre_json_dictionary['layout'] = {
    'title': '{} from {} to {}'.format(', '.join(bird_names), years[0], years[-1]), 
    'xaxis':{'title':'Year'}, 
    'yaxis':{'title':'Number counted'}
}

figure=go.Figure(data=pre_json_dictionary['data'],layout=pre_json_dictionary['layout'])

offline.iplot(figure, filename='top_three')


It's time move this graph to Javascript! You can checkout how I updated my chart in the completed/ directory.

Note: If you haven't done so, I recommend playing with the line options: shape, spline, smoothing, width
...and the marker options: size, opacity


In [ ]:
# This is how we'll display the chart in Javascript.

# Copy the output below, from first '{' to the last '}' 

print(json.dumps(pre_json_dictionary))


If you haven't already done so, [fork my Gist](https://gist.github.com/enactdev/2c45cfa8456fde957898c619ec489cf7)!

(After logging into GitHub, click on 'fork' in the upper right.)

Edit the Gist (button in upper right). Then copy that string above and paste it into the 'plotly_vars.json' file. (Replace the original contents of 'plotly_vars.json')

Refresh your bl.ocks! (Edit the URL, replace 'gist.github.com' with 'bl.ocks.org')


### Next Steps

Optional: Add a thumbnail to your block. Create a .png image that is wider than it is tall, such as 400x250 (width x height), call it 'thumbnail.png' and place it in your Gist. (Don't make it too wide, such as 1200x800.) Easiest way to do this is to create screenshot, crop it, then resize it. 

Optional: That JSON above is not formatted to be as readable as the JSON we are replacing. But, your browser can format it for you! Open the JSON in your browser (you can drag and drop the JSON file into a browser window). If it is not formatted well you may need to install a plugin. Note: This is also a great way to validate your JSON. If there is an error in it then the browser will tell you!

Optional: Make your README more descriptive. Consult this [Markdown Cheat Sheet](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet) and explain where you got the data and how you transformed it. 

Optional: We "hardcoded" (programmer speak) this notebook to use January. At the top, make a new variable 'display_month' and set it to 'January' -- Then replace all other instances of 'January' with your variable and make sure the code works. Then try changing the month. (Note: Not all months are represented in this data, and you will need to make sure your data is both complete and consistant. We determined our January data is complete because we have 11 observations from each year, and determined it is consistant because each observation has a different unit number.) Bonus: Edit the chart title to display the month you set in 'display_month'

### Thank you!
Thanks for going through this tutorial! If you have any questions, find me on the Open Data Delaware Slack.